<!-- # Word Representations -->
# Reprezentarea Cuvintelor

În procesarea limbajului natural, word embeddings (reprezentarea cuvintelor prin proiecții) este termenul folosit pentru reprezentarea cuvintelor ca vectori. Pentru a antrena un model, avem nevoie de date numerice, deci avem nevoie de o modalitate prin care să transformăm un text în vectori de numere încercând să păstrăm cât mai multe informații relevante pentru ce vrem să facem. Așadar, uneori ne interesează relațiile semantice, alteori conținutul lexical șamd.

<!-- In natural language processing, word embedding is the term used for representing a word as a vector. For training a model we need numerical data, which means that we must find a way to represent texts such that we keep as much information as possible considering our current context. This means that sometimes semantic relations will be more important, other times lexical information etc. -->

Prin vectorizarea cuvintelor, reprezentăm fiecare cuvânt ca un număr sau ca o listă de numere. În cazul reprezentărilor dense/continue ale cuvintelor, ideea este să reprezentăm cuvinte similare ca fiind apropiate în spațiul vectorial în care le proiectăm.

<!-- By using word embeddings (vectorization) we can represent each word as a number or a list of numbers that conveys this information such that words that are similar will be closer to each other in the vector space than words that are not. -->

[word2vec ilustrat](https://jalammar.github.io/illustrated-word2vec/) (revenim la word2vec ora viitoare)

# Bag of Words (BoW) / Sac de cuvinte

Pentru situațiile când contextul și ordinea cuvintelor nu este relevantă, ci doar cât de des apar cuvintele, atunci am folosi BoW. E ca și cum am "arunca" toate cuvintele într-un sac (eventual le și amestecăm) iar apoi numărăm de câte ori apare fiecare cuvânt (un fel de vector de frecvență). Ca un caz particular, putem avea BoW binar (un fel de vector de apariții). Aceasta este cam cea mai simplă tehnică de vectorizat un text.

<!-- Imagine a situation where the context of the words is not relevant, only how often they appear. This is where we use bag of words. This approach just throws all words in a bag, maybe shuffles it a bit, then counts how many times each words appears (or if they appear in case of a binary BoW). It is the easiest vectorization method that we will discuss. -->

<center><img src='https://drive.google.com/uc?export=view&id=1v6McR199QkVXvuQmC3FWJ80rSXGTbZUS' width=500></center>

Să luăm exemplul din imagine dat ca exemplu. Fie scriem noi implementarea de la zero, fie folosim [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) gata implementat în scikit-learn:

<!-- Let's take the text from the example. We can either write our own BoW implementation, or we can use the one preimplemented in [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html): -->

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = ['Did you see the fly?', 'The fly will fly with you.']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['did' 'fly' 'see' 'the' 'will' 'with' 'you']
[[1 1 1 1 0 0 1]
 [0 2 0 1 1 1 1]]


`CountVectorizer` are un constructor cu mulți parametri impliciți pe care îi putem suprascrie. De exemplu, dacă vrem doar reprezentare binară (vector de apariții) și să numărăm doar bigrame, am proceda astfel:

<!-- CountVectorizer is a class with predefined parameters. You can always change those parameters, meaning that you can, for example, choose to have a binary representation of bigrams: -->

In [ ]:
text = ['I am not happy.', 'He is very happy']
vectorizer = CountVectorizer(binary=True, ngram_range=(2, 2))
X = vectorizer.fit_transform(text)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['am not' 'he is' 'is very' 'not happy' 'very happy']
[[1 0 0 1 0]
 [0 1 1 0 1]]


N-gramele (la nivel de cuvânt) sunt secvențe de n cuvinte. Ele sunt utile pentru a furniza context, de exemplu pentru a diferenția între _not happy_ și _verry happy_. Le putem folosi fie ca features (reprezentări numerice), fie ca să analizăm setul de date.

<!-- N-grams are sequences of n words. They help us get some context about the text, letting us know the difference between _not happy_ and _very happy_ for example. This can be used as a feature for another representation, or on its own to make assumptions about the dataset. -->

#  Term Frequency - Inverse Document Frequency (Tf-idf)

Doar pentru că un cuvânt apare de multe ori, nu înseamnă că este și relevant dpdv al conținutului (vezi stopwords). De asemenea, stopwords variază de la un domeniu la altul și poate fi anevoios să ne definim manual de fiecare dată liste de stopwords. Dacă de exemplu vrem să ne facem un motor de căutare, ar fi mai relevant un cuvânt care apare des într-un document, însă nu este frecvent întâlnit în celelalte documente (cum se întâmplă în cazul stopwords).

<!-- Just because a word appears often it does not mean that it is necessarily relevant (think about stopwords). If we want to write a search engine for example, it would be more relevant for us to know how often a certain word appears in a document with regards to how common that word generally is. Tf-idf is an algorithm that takes this into account. In other words, a word is important for a given document if it appears many times in this one and rarely in others. -->

Pentru un document dat, repetăm următoarele operații pentru fiecare cuvânt din întreg setul de date:

<!-- We will consider the given document as the current datapoint and repeat the following for each word in the dataset: -->

$$TFIDF = TF * IDF$$
unde:
<!-- $$TF(word, document) = \frac{How\ many\ times\ the\ word\ appears\ in\ the\ document}{Number\ of\ words\ in\ the\ document}$$ -->
$$TF(cuvânt, document) = \frac{\text{#apariții ale cuvântului în document}}{\text{# de cuvinte din document}}$$
și:
$$IDF(cuvânt, Documente) = log(\frac{\text{# de documente din corpus}}{1 + \text{# de documente care conțin cuvântul curent}} + 1)$$

<!-- We use **log** in order to smooth our values for an easier analysis. -->
Folosim **log** pentru a normaliza valorile și pentru a fi mai ușor de analizat.

Pentru implementare, avem [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Reprezentarea va fi o matrice unde fiecare rând corespunde unui document și fiecare coloană corespunde unui cuvânt din tot setul de date:

<!-- For the implementation you can use [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). The output will be a matrix where each row corresponds to a datapoint and each column to a word from the full dataset: -->

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = ['I am not happy.', 'He is very happy']

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['am' 'happy' 'he' 'is' 'not' 'very']
[[0.6316672  0.44943642 0.         0.         0.6316672  0.        ]
 [0.         0.37997836 0.53404633 0.53404633 0.         0.53404633]]


# Reprezentări continue ale cuvintelor

Reprezentările de până acum (BoW, Tf-Idf) au ca dezavantaje
- necesitatea unui număr extrem de mare de elemente pe măsură ce avem texte de dimensiuni mai mari
- nu au un mod de a reprezenta relațiile semantice dintre cuvinte

Pentru a ocupa mai puțină memorie, se folosesc reprezentări cu matrici rare (sparse matrix). Totuși, chiar și așa, aceste reprezentări rare ale cuvintelor nu scalează când dimensiunea vocabularului este foarte mare.

Ca soluții la această problemă, au apărut reprezentările dense (continue) de cuvinte, unde numărul de dimensiuni ale proiecțiilor cuvintelor este mult mai mic.

Dimensiunea unui vocabular poate ajunge la câteva zeci sau sute de mii de cuvinte (și chiar milioane), deci utilizarea unor matrice de astfel de dimensiuni în rețele neurale devine extrem de costisitoare.

### Pe scurt despre word2vec (2013) și GloVe (2014)

Reprezentările continue sunt pe scurt o matrice utilizată pentru a reduce dimensiunea unui vector. Dacă dimensiunea vocabularului este V și reducem la niște embeddings (reprezentări/proiecții) de dimensiune N, avem o matrice de $V*N$. Inițial, din textul nostru vom reprezenta un cuvânt ca un vector one-hot de dimensiune $V$.

Pentru a obține reprezentarea cuvântului, înmulțim vectorul cu matricea de proiecție (embedding matrix), deci obținem un vector de dimensiune $N$.

De exemplu, dimensiunea vocabularului este de $10^5$ și reducem la niște reprezentări de dimensiune 300. Astfel, în loc să lucrăm cu vectori de dimensiune $10^5$ în rețelele noastre, avem doar vectori de dimensiuni 300, mult mai mici.

### fastText

Cu toate că reprezentările word2vec și GloVE ne rezolvă problemele cu reprezentările rare ale cuvintelor, acestea au o limitare (existentă și înainte) referitoare la modul în care ar trebui tratate cuvintele care nu există în setul de date inițial.

Cuvintele din afara vocabularului (OOVW - out of vocabulary words) nu pot fi reprezentate în spațiul vectorial dat de matricea de proiecție, deoarece nu apar în vocabular. Câteva metode de remediere:
- cel mai simplu, adăugăm în vocabular un cuvând UNK (unknown - necunoscut)
- folosim stemming sau lematizare în speranța că vom găsi un cuvânt similar în vocabularul inițial
- căutăm sinonime sau alte cuvinte similare (un pic ironic, nu-i așa? 🙂)

Chiar și așa, tot nu vom avea reprezentări pentru cuvinte inventate sau nou apărute în limbaj.

[fastText](https://fasttext.cc/), apărut în 2016, rezolvă această problemă prin utilizarea de n-grame la nivel de caracter. Astfel, chiar dacă avem un cuvânt necunoscut, se poate încerca obținerea unei reprezentări prin combinarea reprezentărilor n-gramelor la nivel de caracter din care e format acest cuvânt.

### Reprezentări contextuale ale cuvintelor

Reprezentările clasice word2vec, GloVe și fastText au ca dezavantaj utilizarea unei reprezentări fixe ale cuvintelor învățate, indiferent de context. De exemplu, în propoziția
```
Pe cer e un nor, eu cer un marker color.
```
Cuvântul `cer` apare de două ori, dar are înțelesuri diferite: prima dată este substantiv, a doua oară este verb.

Reprezentările continue clasice de cuvinte ar furniza un singur vector pentru acest cuvânt, deși semantic nu există vreo legătură.

Pentru a rezolva această problemă, în 2018 au apărut reprezentările contextuale ale cuvintelor. Astfel, în exemplul anterior, vor exista două reprezentări vectoriale pentru un singur cuvânt pe baza contextului.

Exemple de reprezentări contextuale: BERT, ELMo, GPT-2.

Vezi https://ai.stanford.edu/blog/contextual/ (și cursurile de la masterul de NLP) pentru mai multe detalii.

Asemănător cu fastText, și aceste modele folosesc tokenizări la nivel de "sub-cuvinte" (un fel de n-grame la nivel de caracter) pentru a reprezenta mai ușor cuvinte necunoscute.

Trebuie totuși menționat că și în acest caz este posibil să avem OOV sub-words, însă pentru texte pe subiecte generale este mult mai rar. Probleme de OOV în prezent pot apărea dacă folosim modele antrenate pe alte limbi, din alte domenii sau cu domenii specializate cu mulți termeni de nișă (de exemplu în domeniul biologic sau medical).

Cam acesta este stadiul în care ne aflăm în prezent (2024) referitor la reprezentările cuvintelor.

### Alte direcții

În alte contexte, este posibil să nu avem nevoie de granularitate la fel de mare în reprezentarea textelor. Astfel, există și reprezentări la nivel de propoziție (sent2vec) sau document (doc2vec). Aceste reprezentări au fost adaptate și în contexte mai noi (de exemplu [sentence transformers](https://www.sbert.net/)). Mai multe detalii la masterul de NLP 🙂


# Word2vec

Word2vec a fost una dintre cele mai cunoscute tehnici de reprezentare a cuvintelor înainte de Transformer în 2017 ([arxiv](https://arxiv.org/pdf/1706.03762.pdf), [NeurIPS](https://dl.acm.org/doi/pdf/10.5555/3295222.3295349))/BERT în 2018 ([arxiv](https://arxiv.org/pdf/1810.04805.pdf), [NAACL](https://aclanthology.org/N19-1423.pdf)). Word2vec a apărut în 2013 ([1](https://arxiv.org/pdf/1310.4546.pdf), [2](https://arxiv.org/pdf/1310.4546.pdf), [3](https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)), ideea fiind să folosească o rețea neurală cu un singur strat ascuns antrenată pe fiecare cuvânt în mod independent având ca obiectiv să "apropie" cuvintele similare în spațiul vectorial și să "îndepărteze" cuvintele irelevante.

<!-- Word2vec was one the most popular embedding technique used before the rise of the Transformer in [2017](https://arxiv.org/pdf/1706.03762.pdf). It was originaly published in 2013 ([\[1\]](https://arxiv.org/pdf/1310.4546.pdf), [\[2\]](https://arxiv.org/pdf/1310.4546.pdf)) and it consists of a shallow neural network (with only one hidden layer) trained on each word from a text independently such that similar words are closer to eachother in the vector space (and unrelated words are further). -->

Ideea de bază are origini mult mai vechi în filozofie:
- _You shall know a word by the company it keeps_ ("Ar trebui să înțelegi un cuvânt pe baza vecinilor săi" - John Rupert Firth, 1957, A synopsis of linguistic theory)
- _The meaning of a word is its use in the language_ ("Semnificația unui cuvânt este folosirea lui în limbaj" - Ludwig Wittgenstein, 1953, Philosophical Investigations).

Ideea este că ne folosim de contextul în care apare un cuvânt pentru a calcula similarități între cuvintele dintr-un text. Folosim numeroase contexte pe post de set de date de antrenare, apoi punem modelul să prezică reprezentările cuvintelor țintă. Modelul word2vec poate folosi unul dintre următorii algoritmi:
- Continuous Bag of Words (CBoW - "sac de cuvinte continue"): folosim un context pentru a prezice cuvântul din "mijloc"; merge mai bine pe seturi mici de date
- Skip-Gram: folosim un cuvânt pentru a prezice contextul din jurul său; merge mai bine pe caz general (pentru cuvinte rare)

<!-- It all starts from the quote: _You shall know a word by the company it keeps_. The idea is that we can use the context of a word to compute the similarity between different words in our text, use this as a training dataset and create a prediction model the works as an embedding for the words we have in our corpus. The model can use one of the following algorithms:
- Continuous Bag of Words (CBoW): use the context window around a word to predict the word; better for small datasets
- Skip-Gram: use a target word to predict the context around it; better at generalization (for rare words) -->

<img src= "https://wiki.pathmind.com/images/wiki/word2vec_diagrams.png" width="500" height="300">



<!-- [A more in depth explanation with code](https://www.tensorflow.org/text/tutorials/word2vec) -->
[Explicație detaliată cu cod](https://www.tensorflow.org/text/tutorials/word2vec)

### Continuous Bag-of-Words (CBoW) / Sac de cuvinte continue

Spre deosebire de modelul clasic BoW, CBoW ia în calcul contextul din jurul unui cuvânt fixat folosind o fereastră de context (context window).

De exemplu, dacă alegem textul _The fly will fly with you_ și o fereastră de dimensiune 1, algoritmul se va uita la un cuvânt înainte și un cuvânt după, generând următoarea secvență de perechi (_context_, _cuvânt țintă_):

<!-- Unlike the BoW model, CBoW takes into account the context around a certain word by using a context window. -->


<!-- For example, if you choose the text _The fly will fly with you._ and the window size 1, it will look at exactly 1 word before and after each word in the text, generating the following sequence of (_context_, _target_) pairs: -->

$$([the, will], fly), ([fly, fly], will), ([will, with], fly), ([fly, you], with)$$

Acestea sunt informațiile folosite de model la antrenare pentru a prezice cel mai probabil cuvânt dându-se un anumit context.

<!-- This is the information on which we will train our model to predict the most probable word in a given context. -->

### Skip-Gram

Modelul Skip-Gram funcționează pe dos decât CBoW: dat fiind un cuvânt țintă, să se prezică cel mai probabil context. Pentru a realiza acest lucru, antrenăm o rețea cu un strat ascuns să prezică probabilitatea ca un cuvânt _y_ să apară lângă un cuvânt _x_ într-un text la întâmplare. Stratul ascuns este folosit ca reprezentarea vectorială a unui cuvânt țintă. Distanța în spațiul vectorial dintre 2 cuvinte ar fi mai mică dacă acele cuvinte apar în contexte similare.

<!-- The Skip-Gram Model works the other way around: given a target word, it aims to predict the context around it. In order to do this, you can train a neural network with one hidden layer for a simple task: to predict the chance of having word _y_ really close to word _x_ in a random text. Then you use this layer as the vector representation of the given word, thus making sure that the vector distance between any 2 words is closer if they are more similar and larger if they are not. -->

### Antrenarea unui model
<!-- ### Training a model -->

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

embedding = Word2Vec(
    sentences=common_texts,   # the list of sentences, where each sentence is given as a list of words (processed or not processed)
    vector_size=100,          # the number of features in the vectorized representation
    window=7,                 # the context window
    min_count=3,              # the minimum number of times a word should appear in our dataset in order to be counted
    sg=1                      # sg=1 means skip-gram is used, sg=0 means CBOW is used
)

In [ ]:
embedding.wv.key_to_index

{'system': 0, 'graph': 1, 'trees': 2, 'user': 3}

In [ ]:
import pandas as pd

df = pd.DataFrame(
    [embedding.wv.get_vector(word) for word in embedding.wv.key_to_index.keys()],
    index=embedding.wv.key_to_index
  )

df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
system,-0.000536,0.000236,0.005103,0.009009,-0.009303,-0.007117,0.006459,0.008973,-0.005015,-0.003763,...,0.001631,0.000190,0.003474,0.000218,0.009619,0.005061,-0.008917,-0.007042,0.000901,0.006393
graph,-0.008620,0.003666,0.005190,0.005742,0.007467,-0.006168,0.001106,0.006047,-0.002840,-0.006174,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
trees,0.000095,0.003077,-0.006813,-0.001375,0.007669,0.007346,-0.003673,0.002643,-0.008317,0.006205,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
user,-0.008243,0.009299,-0.000198,-0.001967,0.004604,-0.004095,0.002743,0.006940,0.006065,-0.007511,...,-0.007426,-0.001064,-0.000795,-0.002563,0.009683,-0.000459,0.005874,-0.007448,-0.002506,-0.005550


In [ ]:
embedding.wv.most_similar('system')

[('graph', -0.01083916611969471),
 ('trees', -0.05234673246741295),
 ('user', -0.111670583486557)]

### Încărcarea unui model preantrenat
<!-- ### Loading a pretrained model -->

[Informații despre date și modele](https://github.com/piskvorky/gensim-data)
<!-- [Info about data and models](https://github.com/piskvorky/gensim-data) -->

[Exemple de utilizare](https://radimrehurek.com/gensim/models/word2vec.html)
<!-- [Examples on how to use](https://radimrehurek.com/gensim/models/word2vec.html) -->

In [ ]:
import gensim.downloader as api

api.info()

In [ ]:
model = api.load("word2vec-google-news-300")

[=================---------------------------------] 35.9% 597.3/1662.8MB downloaded

In [ ]:
model.most_similar('system')

[('systems', 0.7227916717529297),
 ('sytem', 0.7129376530647278),
 ('sys_tem', 0.5871982574462891),
 ('System', 0.5275423526763916),
 ('mechanism', 0.5058810114860535),
 ('sysem', 0.5027822852134705),
 ('systen', 0.49969804286956787),
 ('system.The', 0.49599188566207886),
 ('sytems', 0.4949610233306885),
 ('computerized', 0.47604817152023315)]

In [ ]:
model.similarity('system', 'graph')

0.09396098

### Fine-tuning (finisare) pentru modelul anterior:
<!-- ### Fine-tuning our model: -->


In [ ]:
model.train(common_texts, total_examples=4, epochs=1)

Other cool stuff:

In [ ]:
model.most_similar(positive=["king", "woman"], negative=["man"])

[('queen', 0.7118193507194519),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321839332581),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593831062317),
 ('monarchy', 0.5087411999702454)]

[And less cool stuff:](https://arxiv.org/pdf/1607.06520.pdf)

In [ ]:
model.most_similar(positive=["computer_programmer", "woman"], negative=["man"])

[('homemaker', 0.5627118945121765),
 ('housewife', 0.5105047225952148),
 ('graphic_designer', 0.505180299282074),
 ('schoolteacher', 0.497949481010437),
 ('businesswoman', 0.493489146232605),
 ('paralegal', 0.49255111813545227),
 ('registered_nurse', 0.4907974898815155),
 ('saleswoman', 0.4881627559661865),
 ('electrical_engineer', 0.4797725975513458),
 ('mechanical_engineer', 0.4755399227142334)]

Bias is still an unsolved problem in Machine Learning. Do you know any other popular examples of bias?

# Principal Component Analysis (PCA) / Analiza componentelor principale

PCA este un algoritm de reducere a numărului de dimensiuni -- poate fi util pentru a vizualiza date cu sute de dimensiuni într-un spațiu 2D sau 3D. De exemplu, pentru a observa distanța dintre proiecțiile unor cuvinte, avem:

<!-- PCA is a dimensionality reduction algorithm -- meaning that we can use it to visualise our data in 2D or 3D. Here is an example of how you can use it to see the distance between embeddings in 2D: -->

In [ ]:
from sklearn.decomposition import PCA

text = ['system', 'graph', 'trees', 'user']
embeddings = [model[word] for word in text]

pca = PCA(n_components=2)
pca.fit(embeddings)
vectors_2d = pca.transform(embeddings)

Interfața din sklearn este asemănătoare ca cea a unui model de ML. După ce antrenăm modelul, putem pune componentele pe un grafic cu matplotlib:

<!-- We can train it the same way we would a normal ML model, and visualize the results using, for example, a plotting library like matplotlib: -->

In [ ]:
import matplotlib.pyplot as plt

x = [v[0] for v in vectors_2d]
y = [v[1] for v in vectors_2d]

fig, ax = plt.subplots()
ax.scatter(x, y)

for i, txt in enumerate(text):
    ax.annotate(txt, (x[i], y[i]))

plt.show()

## Global Vectors (GloVe) / Vectori globali

Word2Vec se bazează pe statistici locale (apariții la nivel de propoziție). [GloVe](https://nlp.stanford.edu/projects/glove/) ia în calcul statistici globale, ceea ce poate fi util pentru seturi de date mici, nefiind nevoie de multe date de antrenare.

Modelul numără toate perechile "cuvânt1 cuvânt2 ..." (pentru un context de dimensiune x considerăm cuvinte cu distanța cel mult x între ele) și reține informația într-o matrice de număr de apariții (co-occurrence matrix):

<!-- While Word2Vec is based only on local statistics (the occurence of words at
a single-sentence level) [GloVe](https://nlp.stanford.edu/projects/glove/) incorporates global statistics methods. This makes it better suited for smaller datasets, as it does not need as much training data. -->

<!-- The model counts all "word1 word2 ..." pairs (for a context window of x we consider words that have at most distance x between them) and keeps the information in a co-occurrence matrix: -->

<center><img src='https://drive.google.com/uc?export=view&id=1pnX1lPdQItUauHp9W8xJlx8q2lgTe4cJ' width=500></center>

După aceea, se calculează probabilitatea ca un cuvânt să fie mai aproape de alt cuvânt pe baza acestei matrice:

<!-- Afterwards, it computes the probability that a word will be closer to another one based on this matrix: -->
$$P(j | i) = \frac{X_{ij}}{X_i}$$
<!-- where: -->
unde:
$$P(j | i) = \text{probabilitatea să avem cuvântul j dacă avem cuvântul i}$$
$$X_{ij} = \text{de câte ori apare cuvântul j în contextul cuvântului i}$$
$$X_i = \sum_k X_{ik} = \text{numărul total de cuvinte care apar în contextul cuvântului i}$$

<!-- $$P(j | i) = the\ probability\ of\ word\ j\ given\ i$$
$$X_{ij} = how\ many\ times\ word\ j\ appears\ in\ the\ context\ of\ i$$
$$X_i = \sum_k X_{ik} = sum\ of\ how\ many\ times\ words\ appear\ in\ the\ context\ of\ i$$ -->

Pe baza acestor calcule, ar trebui să putem determina relații între cuvinte:

<!-- Based on this we should be able to infer relations between words: -->

<center><img src='https://nlp.stanford.edu/projects/glove/images/table.png' width=500></center>

Observăm că _solid_ are legătură cu _ice_, dar nu și cu _steam_, pe când _gas_ are legătură cu _steam_, dar nu și cu _ice_ (probabilități condiționale foarte mari vs foarte mici). _Water_ și _fasion_ sunt fie strâns corelate cu _ice_ și _steam_ împreună, fie sunt complet fără legătură.

<!-- Notice how _solid_ is related to _ice_ but not _steam_, while _gas_ is related to _steam_ but not _ice_ (very large vs. very small conditional values). _Water_ and _fashion_ on the other hand are either highly related to both or completely unrelated. -->

Mai multe detalii se regăsesc în [articol](https://aclanthology.org/D14-1162.pdf).

<!-- Some more computation will bring us to the regression model that is now used for this model. If you want to learn more you can check [the paper](https://aclanthology.org/D14-1162.pdf). -->

### Utilizarea GloVe
<!-- ### Using GloVe -->

Putem încărca un model GloVe preantrenat folosind biblioteca gensim (sau alte resurse):
<!-- We can load a pretrained GloVe model using the gensim library (or other resources): -->

In [ ]:
import gensim.downloader as api

model = api.load("glove-twitter-100")

[==================================================] 100.0% 387.1/387.1MB downloaded


Pentru a calcula reprezentările cuvintelor (word embeddings) sau pentru alte similarități între cuvinte, la fel ca în cazul Word2Vec:
<!-- And use it to compute the word embeddings (or do all other similarity functions that we saw for Word2Vec): -->

In [ ]:
model['system']

array([ 0.43887 ,  0.32601 , -0.28524 , -0.08248 ,  0.43643 ,  0.75065 ,
        0.093945, -0.72626 ,  0.32297 , -0.37128 , -0.23306 ,  0.35499 ,
       -3.1764  ,  0.015004,  0.69725 , -0.15256 ,  0.025449, -0.058944,
        0.20002 , -0.61298 , -0.79661 ,  0.53051 ,  0.64765 ,  0.90153 ,
       -0.27407 ,  0.52871 ,  0.39344 ,  0.56076 ,  0.31942 ,  0.83347 ,
       -0.53268 , -1.0166  , -0.25328 , -0.17347 ,  0.68794 ,  0.25902 ,
        0.42864 ,  0.3844  , -0.071415, -0.026013, -0.42733 ,  0.58874 ,
       -0.30061 , -0.18357 ,  0.21158 , -0.72648 , -0.48477 ,  0.43527 ,
       -0.37412 , -0.48493 ,  0.26264 ,  0.21684 , -0.8822  ,  0.57925 ,
       -0.54    ,  0.7147  , -0.33133 , -0.44715 , -0.40713 , -0.014364,
       -0.083808,  0.45569 , -0.094374,  0.56057 ,  0.65446 , -0.45768 ,
        0.2522  ,  0.34328 , -0.061001, -0.4899  ,  0.3342  ,  0.41277 ,
       -0.55403 ,  0.30807 ,  0.22867 , -0.53921 ,  0.16439 ,  0.021561,
        0.15131 , -0.70287 ,  1.4152  ,  0.83387 , 

Pentru a antrena un model de la zero:
<!-- Or you can train your own model from scratch: -->

In [ ]:
from glove import Corpus, Glove

corpus = Corpus()
corpus.fit(common_texts, window=4)

glove = Glove(no_components=4, learning_rate=0.1)
glove.fit(corpus.matrix, epochs=10, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model.txt')

# Principal Component Analysis (PCA)

PCA is a dimensionality reduction algorithm -- meaning that we can use it to visualise our data in 2D or 3D. Here is an example of how you can use it to see the distance between embeddings in 2D:

In [ ]:
from sklearn.decomposition import PCA

text = ['system', 'graph', 'trees', 'user']
embeddings = [model[word] for word in text]

pca = PCA(n_components=2)
pca.fit(embeddings)
vectors_2d = pca.transform(embeddings)

We can train it the same way we would a normal ML model, and visualize the results using, for example, a plotting library like matplotlib:

In [ ]:
import matplotlib.pyplot as plt

x = [v[0] for v in vectors_2d]
y = [v[1] for v in vectors_2d]

fig, ax = plt.subplots()
ax.scatter(x, y)

for i, txt in enumerate(text):
    ax.annotate(txt, (x[i], y[i]))

plt.show()

## Exerciții

1. Scrie propria implementare de Bag of Words de la zero. Ar trebui să furnizeze atât reprezentări binare, cât și reprezentări pentru numărul de apariții.
2. Implementează Tf-Idf de la zero. Definește oricâte funcții ajutătoare crezi necesare.
3. Vezi distanța dintre niște cuvinte în 2D folosind PCA (sau altă modalitate de reducere a dimensionalității).
4. Creați perechi de (context, cuvânt_țintă) și antrenați o rețea neurală folosind skip-gram sau continuous bag of words. Ar trebui să etichetați fiecare cuvânt cu un ID unic și să folosiți padding pentru antrenarea cuvintelor din capete (context "dummy" în stânga/dreapta).
5. Vizualizați distanțele dintre câteva cuvinte în 2D folosind PCA (sau alte tehnici de reducere a numărului de dimensiuni).
6. Comparați reprezentările de cuvinte în diverse moduri, de exemplu timp de antrenare, cel mai similar cuvânt pentru cuvântul X, distanțe în spațiul 2D, acuratețea cu SVM etc. Comparați implementările voastre cu implementările furnizate de biblioteci.